# Guide pratique : entraînement PhysAE avec des configurations fixesCe carnet illustre comment préparer les données, sélectionner un backbone, ajuster les hyperparamètres d'entraînement et recueillir les métriques pour chaque variante de modèle disponible dans PhysAE. Toutes les cellules peuvent être exécutées sur CPU ou GPU selon les ressources disponibles.

## 1. Préparer l'environnement1. Créez et activez un environnement virtuel (optionnel mais recommandé).2. Installez PhysAE et ses dépendances :   ```bash   pip install -e .[notebooks]   ```3. Si vous utilisez un GPU, installez la version de PyTorch compatible avec votre pilote CUDA. Consultez <https://pytorch.org/> pour la commande exacte.Ce carnet suppose que vous exécutez les commandes depuis la racine du dépôt PhysAE.

In [ ]:
import torchfrom pathlib import Pathfrom pprint import pprintfrom physae.factory import build_data_and_modelfrom physae.config_loader import load_data_config, load_stage_config, merge_dictsfrom physae.training import train_stage_customROOT = Path.cwd().resolve()CONFIG_DIR = ROOT / 'physae' / 'configs'print(f'Utilisation des configurations à partir de {CONFIG_DIR}')print(f'PyTorch détecte CUDA ? {torch.cuda.is_available()}')

## 2. Explorer les configurations YAMLLes fichiers YAML définissent les plages de génération des données et les paramètres d'entraînement par défaut. Visualisons les éléments clés :

In [ ]:
DATA_CONFIG = load_data_config(None, name='default')print('Clés disponibles dans la configuration de données :')print(sorted(DATA_CONFIG.keys()))STAGE_A = load_stage_config('A')print('\nParamètres principaux du stage A :')pprint({k: STAGE_A[k] for k in ['epochs', 'base_lr', 'refiner_lr', 'delta_scale']})print('\nEspace de recherche Optuna associé (non utilisé dans ce carnet) :')pprint(STAGE_A.get('optuna', {}))

## 3. Construire jeux de données et modèleLa fonction `build_data_and_model` assemble un modèle PhysAE et les loaders données en combinant la configuration YAML et vos surcharges. Nous définissons ci-dessous une fonction utilitaire qui réduit la taille des jeux pour des essais rapides, tout en laissant la possibilité de modifier chaque paramètre.

In [ ]:
from dataclasses import dataclassfrom typing import Dict, Any, Tuple@dataclassclass DemoSetup:    model: Any    loaders: Tuple[Any, Any]    metadata: Dict[str, Any]    stage_params: Dict[str, Any]def build_demo_setup(    *,    stage: str = 'A',    encoder_name: str = 'efficientnet',    encoder_params: Dict[str, Any] | None = None,    refiner_name: str | None = None,    refiner_params: Dict[str, Any] | None = None,    data_overrides: Dict[str, Any] | None = None,    stage_overrides: Dict[str, Any] | None = None,    seed: int = 1234,) -> DemoSetup:    """Prépare un modèle PhysAE et ses loaders pour des essais rapides."""    encoder_params = dict(encoder_params or {})    refiner_name = refiner_name or encoder_name    refiner_params = dict(refiner_params or {})    data_cfg_updates = {        'n_train': 1024,        'n_val': 256,        'batch_size': 32,        'seed': seed,        'model': {            'encoder': {                'name': encoder_name,                'params': encoder_params,            },            'refiner': {                'name': refiner_name,                'params': refiner_params,            },        },    }    if data_overrides:        data_cfg_updates = merge_dicts(data_cfg_updates, data_overrides)    model, loaders, metadata = build_data_and_model(        config_overrides=data_cfg_updates,    )    stage_cfg = load_stage_config(stage)    stage_cfg['epochs'] = min(stage_cfg.get('epochs', 10), 5)    stage_cfg.setdefault('enable_progress_bar', True)    if stage_overrides:        stage_cfg = merge_dicts(stage_cfg, stage_overrides)    return DemoSetup(        model=model,        loaders=loaders,        metadata=metadata,        stage_params=stage_cfg,    )

## 4. Choisir un backbonePhysAE propose quatre variantes principales. Le tableau suivant rappelle leurs caractéristiques et les contextes d'usage recommandés.

In [ ]:
import pandas as pdbackbones = [    {'Backbone': 'efficientnet', 'Points forts': 'Rapide, robuste', 'Usage': 'Référence CPU/GPU'},    {'Backbone': 'efficientnet_large', 'Points forts': 'Capacité accrue', 'Usage': 'Exploration GPU'},    {'Backbone': 'efficientnet_v2', 'Points forts': 'Convergence rapide', 'Usage': 'Finetuning'},    {'Backbone': 'convnext', 'Points forts': 'Performances élevées', 'Usage': 'Ressources GPU'},]display(pd.DataFrame(backbones))

## 5. Lancer un entraînement minimalLa fonction suivante entraîne un stage donné sur CPU ou GPU en affichant les métriques. Réglez `RUN_TRAINING = True` pour exécuter réellement l'entraînement. Les paramètres d'entrée permettent de modifier chaque étape : choix du backbone, du raffineur, des plages de données et des hyperparamètres de stage.

In [ ]:
from typing import OptionalRUN_TRAINING = False  # Passez à True pour lancer les entraînementsDEFAULT_ACCELERATOR = 'gpu' if torch.cuda.is_available() else 'cpu'def run_training_demo(    stage: str = 'A',    *,    encoder_name: str = 'efficientnet',    encoder_params: Dict[str, Any] | None = None,    refiner_name: Optional[str] = None,    refiner_params: Dict[str, Any] | None = None,    data_overrides: Dict[str, Any] | None = None,    stage_overrides: Dict[str, Any] | None = None,    accelerator: Optional[str] = None,) -> None:    setup = build_demo_setup(        stage=stage,        encoder_name=encoder_name,        encoder_params=encoder_params,        refiner_name=refiner_name,        refiner_params=refiner_params,        data_overrides=data_overrides,        stage_overrides=stage_overrides,    )    train_loader, val_loader = setup.loaders    params = dict(setup.stage_params)    params['stage_name'] = stage    params['accelerator'] = accelerator or DEFAULT_ACCELERATOR    params.setdefault('trainer_kwargs', {})    params['trainer_kwargs'].setdefault('log_every_n_steps', 5)    print('Résumé des hyperparamètres :')    pprint({k: params[k] for k in ('epochs', 'base_lr', 'refiner_lr', 'delta_scale')})    print(f"Accélérateur : {params['accelerator']}")    print(f"Taille lot : {train_loader.batch_size}, steps/train : {len(train_loader)}")    if RUN_TRAINING:        _, metrics = train_stage_custom(            setup.model,            train_loader,            val_loader,            **params,            return_metrics=True,        )        print('Métriques de validation :')        pprint(metrics)    else:        print('Entraînement non lancé (RUN_TRAINING=False).')        print('Activez RUN_TRAINING pour obtenir les métriques.')

### Exemple : EfficientNet sur CPUCe scénario limite les échantillons et les époques pour un test rapide sur CPU.

In [ ]:
run_training_demo(    stage='A',    encoder_name='efficientnet',    data_overrides={        'n_train': 512,        'n_val': 128,        'batch_size': 16,    },    stage_overrides={        'epochs': 3,        'base_lr': 2e-4,        'delta_scale': 0.12,        'accelerator': 'cpu',    },)

### Exemple : ConvNeXt sur GPUSur une machine équipée d'un GPU, augmentez légèrement la capacité et la durée d'entraînement.

In [ ]:
run_training_demo(    stage='A',    encoder_name='convnext',    encoder_params={'width_mult': 0.75},    data_overrides={        'n_train': 2048,        'n_val': 512,        'batch_size': 32,    },    stage_overrides={        'epochs': 6,        'base_lr': 1.5e-4,        'delta_scale': 0.1,    },    accelerator='gpu' if torch.cuda.is_available() else 'cpu',)

### Boucle sur tous les backbonesUtilisez la cellule ci-dessous pour générer automatiquement les configurations d'entraînement pour chaque backbone. Cela permet de comparer facilement les paramètres.

In [ ]:
for name in ['efficientnet', 'efficientnet_large', 'efficientnet_v2', 'convnext']:    print(f"\n--- Préparation du backbone {name} ---")    run_training_demo(        stage='A',        encoder_name=name,        stage_overrides={'epochs': 4},    )

## 6. Ajuster les stages B1/B2 et le fine-tuningAprès le stage A, enchaînez avec les stages B1 et B2 pour raffiner le modèle. Utilisez le paramètre `ckpt_in` pour charger un checkpoint précédent, ou `ckpt_out` pour sauvegarder le modèle entraîné :

In [ ]:
run_training_demo(    stage='B1',    stage_overrides={        'epochs': 4,        'train_refiner': True,        'train_base': False,        'refine_steps': 2,        'ckpt_in': 'path/vers/stage_A.ckpt',  # Remplacez par votre checkpoint        'ckpt_out': 'artifacts/stage_B1.ckpt',    },)

## 7. Bonnes pratiques* **Stabilité** : commencez avec des tailles de lot réduites et augmentez-les progressivement.* **CPU vs GPU** : laissez `accelerator=None` pour l'auto-détection ; forcez `"cpu"` pour déboguer.* **Normalisation** : si vous modifiez les plages d'échantillonnage, vérifiez que les valeurs restent compatibles avec `config.NORM_PARAMS`.* **Suivi des expériences** : stockez les sorties (checkpoints, métriques) dans un dossier dédié par run.* **Extensions** : ajoutez vos backbones via `physae.models.register_encoder` et passez le nom dans la section `model.encoder`.